In [1]:
import os
os.environ['FILEREADER_CACHE'] = '1'

from openpilot.tools.lib.comma_car_segments import get_comma_car_segments_database
from openpilot.selfdrive.car.ford.values import CAR

database = get_comma_car_segments_database()

platforms = [c.value for c in CAR]

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /mnt/c/Users/camer/AppData/Local/Programs/Microsoft VS Code


In [2]:
import random

from opendbc.can.parser import CANParser
from openpilot.tools.lib.logreader import LogReader
from openpilot.selfdrive.car.ford.values import DBC

MAX_SEGS_PER_PLATFORM = 2

msgs = [('Cluster_Info_3_FD1', 0)]

for platform in platforms:
  if platform not in database:
    print(f'Skipping platform: {platform}, no data available')
    continue

  dbc = DBC[platform]['pt']
  if dbc is None:
    print(f'Skipping platform: {platform}, no dbc available')
    continue

  all_segments = database[platform]

  NUM_SEGMENTS = min(len(all_segments), MAX_SEGS_PER_PLATFORM)

  print(f'Got {len(all_segments)} segments for platform {platform}, sampling {NUM_SEGMENTS} segments')

  for segment in random.sample(all_segments, NUM_SEGMENTS):
    can_strings = []
    for msg in LogReader(segment):
      if msg.which() != 'can':
        continue
      can_strings.append(msg.as_builder().to_bytes())

    if can_strings:
      parser = CANParser(dbc, msgs, 0)
      parser.update_strings(can_strings)

      scaling = parser.vl['Cluster_Info_3_FD1']['DISPLAY_SPEED_SCALING']
      offset = parser.vl['Cluster_Info_3_FD1']['DISPLAY_SPEED_OFFSET']
      if scaling != 0 and offset != 0:
        print(f'{scaling=}, {offset=}')
        break

Got 287 segments for platform FORD BRONCO SPORT 1ST GEN, sampling 2 segments
Got 137 segments for platform FORD ESCAPE 4TH GEN, sampling 2 segments
Got 1041 segments for platform FORD EXPLORER 6TH GEN, sampling 2 segments
Got 5 segments for platform FORD F-150 14TH GEN, sampling 2 segments
scaling=100.5, offset=1.0
Got 56 segments for platform FORD FOCUS 4TH GEN, sampling 2 segments
Got 637 segments for platform FORD MAVERICK 1ST GEN, sampling 2 segments
Got 3 segments for platform FORD F-150 LIGHTNING 1ST GEN, sampling 2 segments
scaling=100.5, offset=1.0
Got 3 segments for platform FORD MUSTANG MACH-E 1ST GEN, sampling 2 segments
scaling=100.5, offset=1.0
